In [2]:
# download data
!gdown --id 1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86
#download labels
!gdown --id 1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH
!gdown --id 1wOdja-ezstMEp81tX1a-EYkFebev4h7D

Downloading...
From: https://drive.google.com/uc?id=1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86
To: /content/fairface-img-margin025-trainval.zip
100% 578M/578M [00:03<00:00, 179MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH
To: /content/fairface_label_train.csv
100% 3.79M/3.79M [00:00<00:00, 59.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wOdja-ezstMEp81tX1a-EYkFebev4h7D
To: /content/fairface_label_val.csv
100% 448k/448k [00:00<00:00, 66.2MB/s]


In [3]:
# unzip data
!unzip -qq "/content/fairface-img-margin025-trainval.zip" -d "/content/fairface/"

In [4]:
import pandas as pd 
import numpy as np
import copy

In [5]:
df = pd.read_csv("/content/fairface_label_train.csv")
train_df = copy.deepcopy(df[["file","gender"]])
train_df.gender = train_df.gender.replace({"Male":0,"Female":1})

In [6]:
df.gender.value_counts()

Male      45986
Female    40758
Name: gender, dtype: int64

In [7]:
train_df = train_df.sample(n = 4000 , random_state = 42)
train_df.head()

,file,gender
47413,train/47414.jpg,0
52263,train/52264.jpg,0
21583,train/21584.jpg,1
32129,train/32130.jpg,0
63300,train/63301.jpg,0


In [8]:
train_df.gender.value_counts()

0    2195
1    1805
Name: gender, dtype: int64

In [9]:
val_df = pd.read_csv('/content/fairface_label_val.csv')
val_df = val_df[["file","gender"]]
val_df.gender = val_df.gender.replace({"Male":0,"Female":1})

val_df.head()

,file,gender
0,val/1.jpg,0
1,val/2.jpg,1
2,val/3.jpg,0
3,val/4.jpg,1
4,val/5.jpg,0


In [10]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111


In [11]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "/content/fairface/"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

# Model Training and Validation Code
The train_model function handles the training and validation of a given model. As input, it takes a PyTorch model, a dictionary of dataloaders, a loss function, an optimizer, a specified number of epochs to train and validate for, and a boolean flag for when the model is an Inception model. The is_inception flag is used to accomodate the Inception v3 model, as that architecture uses an auxiliary output and the overall model loss respects both the auxiliary output and the final output, as described here. The function trains for the specified number of epochs and after each epoch runs a full validation step. It also keeps track of the best performing model (in terms of validation accuracy), and at the end of training returns the best performing model. After each epoch, the training and validation accuracies are printed.

In [12]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    # Transfer learning taking the weight and arctecture of old model 
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                # set flag in train mode (activate drop out layers in train) turn neurons off for regularization
                model.train()  # Set model to training mode
            else:
                # shut down that drop out layers that i don't need 
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            #DATA LOADER
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) # move the data from desk to ram
                labels = labels.to(device) #

                # zero the parameter gradients
                # make the gradiant equal zero at every mini patch cuz its new pics u train on 
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        #Aux classifier
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        #forward probagation
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics for performance
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model make a check point in ram (correct it and put it in disk)
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()
    # calculate time taken in for the epoch
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [13]:
#Freez weights if u use feature extration
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [14]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
#Data loader to get data while ending of training while training on patch
# some of famous data loader "Dataset folder" -> you should have every class in each folder
#our data is diffrent so we will write the data loader
#u can use fast AI u can get another dataloader
import os
from torch.utils.data import Dataset
from torchvision.io import  read_image

class FacesDataset(Dataset):
  def __init__ (self, annotations_df, img_dir , transform = None, target_transform = None):
    self.img_labels = annotations_df
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_labels)
  
  def __getitem__(self,idx):
    # join directory with file path to have image path
    img_path = os.path.join(self.img_dir , self.img_labels.iloc[idx,0])
    image = read_image(img_path)
    # image label
    label = self.img_labels.iloc[idx,1]
    if self.transform:
      image = self.transform(image)
    if self.target_transform:
      label = self.target_transform(label)
    return image , label



In [16]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ConvertImageDtype(torch.float),

        #transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        #transforms.ToTensor(),
        # mean and variance for 3 channels from image net data set
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ConvertImageDtype(torch.float),
        transforms.Resize(input_size),
        #transforms.CenterCrop(input_size),
        #transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
} 

print("Initializing Datasets and Dataloaders...")

# Create train ing and validation datasets
image_datasets = {x: FacesDataset(dataframe,data_dir,data_transforms[x]) for x,dataframe in zip(['train', 'val'],[train_df,val_df])}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [17]:
# Send the model to GPU
model_ft = model_ft.to(device)

#Gather the parameters to be optimized/updated in this run. if we are 
# finetuning we will be updating all parameters. However , if we are 
# doing feature extract method, we will only update the parameters
# that we have just initializzed , i.e the parameters with requires_grad is True
params_to_update = model_ft.parameters()
print("params to learn:")
if feature_extract:
  params_to_update = []
  for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
      params_to_update.append(param)
      print("\t",name)
else:
  for name,apram in model_ft.named_parameters():
    if param.requires_grad == True:
      print("\t",name)

#observe that all parameters are being optimized 
# stochastic gradient decent
optimizer_ft = optim.SGD(params_to_update , lr=0.001, momentum=0.9)

params to learn:
	 fc.weight
	 fc.bias


In [18]:
#Setup the loss fxn
criterion = nn.CrossEntropyLoss()

#Train and evaluate
model_ft,hist = train_model(model_ft,dataloaders_dict,criterion,optimizer_ft,num_epochs=num_epochs,is_inception = False)

Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.6709 Acc: 0.6350
val Loss: 0.5701 Acc: 0.6994

Epoch 1/14
----------
train Loss: 0.5899 Acc: 0.6935
val Loss: 0.6079 Acc: 0.7017

Epoch 2/14
----------
train Loss: 0.6059 Acc: 0.6985
val Loss: 0.5698 Acc: 0.7166

Epoch 3/14
----------
train Loss: 0.5810 Acc: 0.7040
val Loss: 0.7012 Acc: 0.6463

Epoch 4/14
----------
train Loss: 0.6364 Acc: 0.6893
val Loss: 0.5623 Acc: 0.7216

Epoch 5/14
----------
train Loss: 0.5846 Acc: 0.7127
val Loss: 0.5677 Acc: 0.7220

Epoch 6/14
----------
train Loss: 0.5986 Acc: 0.6965
val Loss: 0.6466 Acc: 0.6942

Epoch 7/14
----------
train Loss: 0.5871 Acc: 0.7075
val Loss: 0.6091 Acc: 0.7091

Epoch 8/14
----------
train Loss: 0.6089 Acc: 0.7023
val Loss: 0.6571 Acc: 0.6659

Epoch 9/14
----------
train Loss: 0.6170 Acc: 0.6920
val Loss: 0.5991 Acc: 0.7129

Epoch 10/14
----------
train Loss: 0.6029 Acc: 0.7003
val Loss: 0.6448 Acc: 0.6746

Epoch 11/14
----------
train Loss: 0.6047 Acc: 0.7065
val Loss: 0.5794 Acc: 0.7123

Epoch 12/14
----------
t